In [1]:
import os
import sys
import pandas as pd
import numpy as np

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2484984158827955123
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1489587404
locality {
  bus_id: 1
}
incarnation: 17065783024194312859
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [2]:
from scipy.io import loadmat
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [3]:
# reading in the test/train data for images and labels.

train_data = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_data.mat''')['train_data']
train_labels = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']
test_data = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\test_data.mat''')['test_data']
test_labels = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\test_list.mat''')['labels']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
print(train_data.shape)
print(train_labels.shape)

(12000, 12000)
(12000, 1)


In [6]:
df = pd.DataFrame(train_data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11990,11991,11992,11993,11994,11995,11996,11997,11998,11999
0,1.000000,0.370212,0.386276,0.372247,0.363282,0.341025,0.270990,0.346724,0.394529,0.358653,...,0.267177,0.357578,0.420985,0.409833,0.276110,0.346354,0.234670,0.212166,0.337528,0.268254
1,0.370212,1.000000,0.454343,0.380271,0.390251,0.280266,0.202480,0.290440,0.306661,0.279821,...,0.177962,0.309295,0.417339,0.379545,0.200530,0.253226,0.155548,0.143595,0.309500,0.177095
2,0.386276,0.454343,1.000000,0.371318,0.401308,0.307094,0.206848,0.295721,0.347681,0.256991,...,0.192064,0.320620,0.390572,0.373608,0.208791,0.279283,0.152269,0.169688,0.287026,0.204373
3,0.372247,0.380271,0.371318,1.000000,0.357756,0.327126,0.271649,0.324782,0.357688,0.290282,...,0.254010,0.353008,0.367482,0.375448,0.302831,0.309191,0.233176,0.188171,0.299682,0.234346
4,0.363282,0.390251,0.401308,0.357756,1.000000,0.259415,0.184212,0.273173,0.338933,0.276645,...,0.175661,0.299105,0.410328,0.354185,0.175331,0.244246,0.166500,0.135681,0.318807,0.171231


In [7]:
labels = [item for label in train_labels for item in label] 
df2 = pd.DataFrame({'label':labels})

pre_split = pd.concat([df, df2], axis=1)
pre_split.head()

,0,1,2,3,4,5,6,7,8,9,...,11991,11992,11993,11994,11995,11996,11997,11998,11999,label
0,1.000000,0.370212,0.386276,0.372247,0.363282,0.341025,0.270990,0.346724,0.394529,0.358653,...,0.357578,0.420985,0.409833,0.276110,0.346354,0.234670,0.212166,0.337528,0.268254,1
1,0.370212,1.000000,0.454343,0.380271,0.390251,0.280266,0.202480,0.290440,0.306661,0.279821,...,0.309295,0.417339,0.379545,0.200530,0.253226,0.155548,0.143595,0.309500,0.177095,1
2,0.386276,0.454343,1.000000,0.371318,0.401308,0.307094,0.206848,0.295721,0.347681,0.256991,...,0.320620,0.390572,0.373608,0.208791,0.279283,0.152269,0.169688,0.287026,0.204373,1
3,0.372247,0.380271,0.371318,1.000000,0.357756,0.327126,0.271649,0.324782,0.357688,0.290282,...,0.353008,0.367482,0.375448,0.302831,0.309191,0.233176,0.188171,0.299682,0.234346,1
4,0.363282,0.390251,0.401308,0.357756,1.000000,0.259415,0.184212,0.273173,0.338933,0.276645,...,0.299105,0.410328,0.354185,0.175331,0.244246,0.166500,0.135681,0.318807,0.171231,1


In [8]:
train, validate = train_test_split(df, test_size = 0.2, stratify=train_labels, random_state=16)

In [9]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,11990,11991,11992,11993,11994,11995,11996,11997,11998,11999
4346,0.322709,0.267931,0.281449,0.297712,0.259242,0.335441,0.349134,0.394268,0.259501,0.293976,...,0.390242,0.365563,0.278592,0.322319,0.319473,0.353347,0.264001,0.372329,0.300654,0.365475
1895,0.334906,0.243707,0.275596,0.306293,0.223655,0.365035,0.372774,0.397690,0.267229,0.328474,...,0.351376,0.336478,0.251471,0.305041,0.367305,0.382617,0.308046,0.337379,0.244758,0.356931
8064,0.281262,0.159415,0.162632,0.281583,0.168625,0.301370,0.362225,0.381443,0.235047,0.311864,...,0.331580,0.317978,0.219133,0.246647,0.379795,0.341016,0.312345,0.325242,0.232786,0.351501
2138,0.341854,0.340799,0.315841,0.341646,0.323526,0.324015,0.291754,0.319159,0.323100,0.322483,...,0.227378,0.350138,0.339472,0.366221,0.262715,0.292098,0.302035,0.219769,0.284869,0.211938
9576,0.372642,0.324510,0.311419,0.393962,0.351303,0.346245,0.291129,0.328887,0.316775,0.301948,...,0.274405,0.366048,0.355431,0.369575,0.298750,0.315565,0.288739,0.181200,0.269478,0.219351


In [10]:
#might have finally figured out how to stratify the image data...

X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, stratify=train_labels, random_state=16)

Note on the data.... X_train X_val are already resized from 0-255 to 0-1. No need to transform these arrays. 

In [16]:
# transform the y_train and y_val to categoricals  ***not sure if this is needed**
y_train_onehot = to_categorical(y_train)
y_val_onehot = to_categorical(y_val)
num_classes = y_val.shape[0]
num_classes

2400

In [11]:
X_train.shape

(9600, 12000)

In [12]:
# using a simple CNN to start

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering('tf')


In [33]:
input_shape = (9600, 12000, 0)

# create the model

model = Sequential()
model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Conv2D(32, (11, 11), strides=4, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Conv2D(16, (11, 11), strides=4, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
     

# Compile model
epochs = 25
lrate = 0.003
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 2398, 2998, 64)    64        
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 599, 749, 64)      0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 148, 185, 32)      247840    
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 37, 46, 32)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 7, 9, 16)          61968     
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 1, 2, 16)          0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 32)                0         
__________

In [34]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=32)
scores = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

ValueError: Error when checking input: expected conv2d_51_input to have 4 dimensions, but got array with shape (9600, 12000)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))